In [10]:
from konlpy.tag import Twitter
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize
from scipy.sparse import dok_matrix
from collections import Counter
import numpy as np
import json
import pickle

twitter = Twitter()

In [ ]:
with open("data/preprocessed_data.pkl", 'rb') as f:
    preprocessed_data = pickle.load(f)

In [57]:
# NMF
K = 10
nmf = NMF(n_components=K, max_iter=1000, alpha=0.1)
W = nmf.fit_transform(preprocessed_data['tdm'])
H = nmf.components_

# 각 토픽별 키워드 출력
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(preprocessed_data['index2voca'][index], end=" ")
    print("\n")

0th topic
대통령 박근혜 청와대 탄핵 파면 삼성동 사저 결정 의원 인용 검찰 헌법재판소 대한 입장 국민 지지자 없다 지난 있는 변호사 

1th topic
켈리 방송사고 웃음 스타 부부 행복 등장 사탕 교수 아이 인터뷰 로버트 부산 영국 방송 동영상 아내 아들 영상 모습 

2th topic
대표 의원 대선 민주당 후보 정당 한국 탄핵 지사 더불어 문재인 바른 자유 경선 국민 주자 탈당 국민의당 국회 말했 

3th topic
집회 탄핵 태극기 경찰 국민 반대 기각 참가자 운동 본부 궐기 위한 오후 단체 행진 대통령 주최 광장 촛불 시위 

4th topic
시간 대선 지원 출마 억원 보통사람 인상 검찰 소환 정부 청춘 황교안 변경 금리 공개 문재인 주택 한국 출시 이상 

5th topic
탄핵 선고 심판 헌재 헌법재판소 결정 재판관 대통령 인용 오전 박근혜 기각 대한 국회 평의 결과 사건 의견 기일 각하 

6th topic
사드 국민 중국 우리 경제 한국 정부 있는 배치 대한 문제 정치 탄핵 미국 상황 결정 대한민국 말했 북한 국가 

7th topic
수사 특검 검찰 결과 혐의 박영수 사실 발표 대통령 최순실 대한 조사 특별검사 사건 삼성 대해 의혹 변호사 내용 뇌물 

8th topic
서울 박근혜 오후 광장 탄핵 광화문 열린 촛불집회 퇴진 행동 인용 헌법재판소 종로구 시민 뉴시스 국민 촛불 참가자 정권 하고 

9th topic
대행 권한 대선 대통령 출마 황교안 재판관 정미 선거 퇴임 헌재 후보 국정 소장 상황 헌법재판소장 선거일 국무회의 지정 있는 



In [56]:
from sklearn.manifold import TSNE

# select random index
selectNum = 1000
randIndex = np.random.choice(W.shape[0], selectNum, replace=False)
randIndex.sort()

tsne = TSNE(n_components=2, init='pca', verbose=1)
W2d = tsne.fit_transform(W[randIndex, :])
topicIndex = [v.argmax() for v in W[randIndex, :]]

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.028s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 47.398952
[t-SNE] Error after 1000 iterations: 0.057213


In [ ]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=800, plot_height=500, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in randIndex],
    'document': [articleList[randInd][:50] for randInd in randIndex],  # 해당 기사
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color',
    fill_alpha=0.7,
    line_alpha=0.7)

# 몇가지 interaction
p.legend.location = "top_left"
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)